In [1]:
        #!pip uninstall langchain
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.llms import OpenAI
import os
import pandas as pd

In [26]:
# Import data from FantasyTracker23 folder
path = 'F1_Fantasy\\FantasyTracker23\\TreatedData2023\\'
df_drivers = pd.read_csv(path + 'F1_2023_Fantasy_Drivers_Standings.csv')
df_constructors = pd.read_csv(path + 'F1_2023_Fantasy_Constructors_Standings.csv')
df_races = pd.read_csv(path + 'F1_2023_Fantasy_Races.csv')
df_prices = pd.read_csv(path + 'F1_2023_Fantasy_DriverAndConstructors_Prices.csv')


In [27]:
df_constructors.drop(columns=['BRA', 'BRA1'], inplace=True)

In [28]:
df_prices

,Name,CurrentPrice,AveragePointsPerMillion,Average Points,CurrentPoints,IsConstructor
0,Hamilton,25.3,0.91,23.00,506.0,False
1,Russell,20.2,0.86,17.45,384.0,False
2,Leclerc,23.5,0.65,15.23,335.0,False
3,Sainz,20.4,0.88,17.91,394.0,False
4,Verstappen,30.0,1.54,46.09,1014.0,False
5,Perez,20.8,1.27,26.32,579.0,False
6,Norris,19.8,1.01,19.91,438.0,False
7,Piastri,15.4,0.82,12.68,279.0,False
8,Alonso,15.8,1.16,18.32,403.0,False
9,Stroll,14.0,0.75,10.50,231.0,False


In [29]:
import pandas as pd

def get_user_input():
    name = input("Enter your name: ")
    budget = input("Enter your budget: ")
    while not budget.isdigit():
        budget = input("Invalid input. Please enter a valid budget: ")
    budget = float(budget)
    constructors = []
    drivers = []
    while len(constructors) < 2:
        constructor = input("Enter a constructor: ")
        if constructor not in constructors:
            if df_prices[(df_prices["Name"] == constructor) & (df_prices["IsConstructor"] == True)]["CurrentPrice"].tolist():
                constructors.append(constructor)
            else:
                print(f"{constructor} is not a valid constructor. Please enter a valid constructor.")
        else:
            print("You have already selected this constructor.")
    while len(drivers) < 5:
        driver = input("Enter a driver: ")
        if driver not in drivers:
            if df_prices[(df_prices["Name"] == driver) & (df_prices["IsConstructor"] == False) & (df_prices["CurrentPrice"] <= budget)]["CurrentPrice"].tolist():
                drivers.append(driver)
                budget -= df_prices[df_prices["Name"] == driver]["CurrentPrice"].tolist()[0]
            else:
                print(f"{driver} is not a valid driver. Please enter a valid driver.")
        else:
            print("You have already selected this driver.")
    return name, budget, constructors, drivers

def make_changes(name, budget, constructors, drivers):
    print(f"Hello {name}! Your budget is {budget}.")
    print("Here is your current team:")
    print(f"Constructors: {constructors}")
    print(f"Drivers: {drivers}")
    while True:
        change = input("Would you like to make any changes? (y/n): ")
        if change.lower() == "n":
            break
        elif change.lower() == "y":
            change_type = input("What would you like to change? (constructor/driver): ")
            if change_type.lower() == "constructor":
                old_constructor = input("Enter the name of the constructor you want to replace: ")
                new_constructor = input("Enter the name of the new constructor: ")
                if new_constructor not in constructors:
                    if df_prices[(df_prices["Name"] == new_constructor) & (df_prices["IsConstructor"] == True)]["CurrentPrice"].tolist():
                        constructors[constructors.index(old_constructor)] = new_constructor
                        print("Constructor updated successfully!")
                    else:
                        print(f"{new_constructor} is not a valid constructor. Please enter a valid constructor.")
                else:
                    print("You have already selected this constructor.")
            elif change_type.lower() == "driver":
                old_driver = input("Enter the name of the driver you want to replace: ")
                new_driver = input("Enter the name of the new driver: ")
                if new_driver not in drivers:
                    if df_prices[(df_prices["Name"] == new_driver) & (df_prices["IsConstructor"] == False) & (df_prices["CurrentPrice"] <= budget)]["CurrentPrice"].tolist():
                        drivers[drivers.index(old_driver)] = new_driver
                        budget -= df_prices[df_prices["Name"] == new_driver]["CurrentPrice"].tolist()[0]
                        print("Driver updated successfully!")
                    else:
                        print(f"{new_driver} is not a valid driver. Please enter a valid driver.")
                else:
                    print("You have already selected this driver.")
            else:
                print("Invalid input. Please enter a valid input.")
        else:
            print("Invalid input. Please enter a valid input.")
    print("Here is your final team:")
    print(f"Constructors: {constructors}")
    print(f"Drivers: {drivers}")

name, budget, constructors, drivers = get_user_input()
make_changes(name, budget, constructors, drivers)


 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
 is not a valid constructor. Please enter a valid constructor.
Hello ! Your budget is 213028.50000000003.
Here is your current team:
Constructors: ['Red Bull', 'Ferrari']
Drivers: ['Verstappen', 'Leclerc', 'Sargeant', 'Piastri', 'Norris']
Here is your final team:
Constructors: ['Red Bull', 'Ferrari']
Drivers: ['Verstappen', 'Leclerc', 'Sargeant', 'Piastri', 'Norris']


In [30]:
widgets.BoundedIntText(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Text:',
    disabled=False
)

# Save the value of the budget slider
budget = widgets.value

print('Budget: ' + str(budget))

NameError: name 'widgets' is not defined

In [ ]:
thing = widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)

# Store output from this widget in a variable
display(thing)


In [ ]:
df_prices

In [ ]:
import ipywidgets as widgets
budget_slider = widgets.IntSlider(
    value=100,
    min=50,
    max=170,
    step=1,
    description='Budget:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# One change of the slider, change value_budget automatically
def on_value_change(change):
    global value_budget
    value_budget = change['new']
    print(value_budget)


budget_slider.observe(on_value_change, names='value')

display(budget_slider)

#### Now for the drivers and constructors. We can use Boolean widgets for this. If they are selected, they are True, if not, they are False.
# If they are selected, remove them from the list of available drivers/constructors and subtract their price from the budget.
# If they are selected then deselected, add them back to the list of available drivers/constructors and add their price back to the budget.

drivers = df_prices[df_prices["IsConstructor"] == False][["Name", 'CurrentPrice']]
constructors = df_prices[df_prices["IsConstructor"] == True][["Name", 'CurrentPrice']]

remaining_budget = value_budget

# Drivers and constructors should be separate. Only 5 drivers can be selected, and only 2 constructors can be selected. No more, no less.
# If the team is within budget, then the user can submit it. If not, then they have to change it.
# Store the team

drivers_widget = widgets.SelectMultiple(
    options=drivers["Name"].tolist(),
    value=[],
    #rows=10,
    description='Drivers',
    disabled=False
)



In [ ]:
drivers = df_prices[df_prices["IsConstructor"] == False][["Name", 'CurrentPrice']]
drivers

In [ ]:
selected_fruits =  thing.value
print(selected_fruits)

In [ ]:
# Check how much is the user team worth
sum([df_prices[(df_prices["Name"] == constructor) & (df_prices["IsConstructor"] == True)]["CurrentPrice"].tolist()[0] for constructor in constructors]) + sum([df_prices[(df_prices["Name"] == driver) & (df_prices["IsConstructor"] == False)]["CurrentPrice"].tolist()[0] for driver in drivers])

In [ ]:
team

In [ ]:
# Get the team of the user, knowing that they can only have 2 constructors and 5 drivers.
# Ask first his budget, and then his team.

# Greet user and ask for name
print("Hello, I am your F1 Fantasy assistant. What is your name?")
name = input()

# Greet user and ask for budget
print("Hello " + name + ", what is your budget?")
budget = input()

while budget.isnumeric() == False:
    print("Please enter a valid budget.")
    budget = input()

budget = float(budget)

# Give all the constructors and their prices, and ask for the user to choose 2 of them
print("Here are the constructors and their prices:")
print(df_prices[['Constructor', 'CurrentPrice']])
print("Please choose 2 constructors, and enter their names separated by a comma.")
constructors = input()

while len(constructors.split(',')) != 2:
    print("Please enter 2 constructors.")
    constructors = input()

constructors = constructors.split(',')

# Give all the drivers and their prices, and ask for the user to choose 5 of them
print("Here are the drivers and their prices:")
print(df_drivers[['Driver', 'Price']])
print("Please choose 5 drivers, and enter their names separated by a comma.")
drivers = input()

while len(drivers.split(',')) != 5:
    print("Please enter 5 drivers.")
    drivers = input()


In [ ]:
# Feed JSON data to the agent
from langchain.document_loaders import DirectoryLoader, TextLoader

#loader = DirectoryLoader('F1_Data', glob='**/*.json', show_progress=True, loader_cls=TextLoader)
loader = DirectoryLoader(path, glob='**/*.csv', show_progress=True, loader_cls=TextLoader)

# loader for pdf
#loader = DirectoryLoader('F1_Data', glob='**/*.pdf', show_progress=True, loader_cls=PDFLoader)

docs = loader.load()
print(len(docs))

from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

index.query(prompt)


In [ ]:
# Feed JSON data to the agent
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import PyPDFLoader

#loader = DirectoryLoader('F1_Data', glob='**/*.json', show_progress=True, loader_cls=TextLoader)
loader_csv = DirectoryLoader('FinalData', show_progress=True, use_multithreading=True)

csv_docs = loader_csv.load()
# loader for pdf
#loader = DirectoryLoader('F1_Data', glob='**/*.pdf', show_progress=True, loader_cls=PDFLoader)

#docs = loader.load()
#print(len(docs))

from langchain.indexes import VectorstoreIndexCreator
#index = VectorstoreIndexCreator().from_loaders([loader_csv, loader_pdf])



In [2]:
# Function to read CSV files from a folder
import os
import pandas as pd
def read_csv_folder(path):
    df = pd.DataFrame()
    for file in os.listdir(path):
        if file.endswith(".csv"):
            df = df.append(pd.read_csv(path + file))
    return df

# File to read ad preprocess the PDF files from a folder
import os
import pandas as pd
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
import re
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

def preprocess_pdf_files_for_LLM(path: str):
    pdf_docs = []
    for file in os.listdir(path):
        if file.endswith(".pdf"):
            pdf_docs.append(os.path.join(path, file))
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    embeddings = OpenAIEmbeddings()
    # embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)
    return vectorstore

In [4]:
pdf_files = preprocess_pdf_files_for_LLM("FinalData\\Articles\\")

In [6]:
pdf_files

In [ ]:
# Feed JSON data to the agent
from langchain.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain_community.document_loaders import PyPDFLoader

#loader = DirectoryLoader('F1_Data', glob='**/*.json', show_progress=True, loader_cls=TextLoader)
loader_csv = DirectoryLoader('FinalData', show_progress=True, use_multithreading=True, loader_cls=CSVLoader, glob='**/*.csv')

csv_docs = loader_csv.load()
# loader for pdf
#loader = DirectoryLoader('F1_Data', glob='**/*.pdf', show_progress=True, loader_cls=PDFLoader)

#docs = loader.load()
#print(len(docs))

from langchain.indexes import VectorstoreIndexCreator
#index = VectorstoreIndexCreator().from_loaders([loader_csv, loader_pdf])



In [ ]:
# Feed pdf_files and csv_docs to the agent
from langchain.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain_community.document_loaders import PyPDFLoader

#loader = DirectoryLoader('F1_Data', glob='**/*.json', show_progress=True, loader_cls=TextLoader)
loader_csv = DirectoryLoader('FinalData', show_progress=True, use_multithreading=True, loader_cls=CSVLoader, glob='**/*.csv')
loader_pdf = PyPDFLoader(file_path="FinalData\\Articles\\")
csv_docs = loader_csv.load()
pdf_docs = loader_pdf.load_and_split()
# loader for pdf
#loader = DirectoryLoader('F1_Data', glob='**/*.pdf', show_progress=True, loader_cls=PDFLoader)


In [ ]:
loader_pdf = DirectoryLoader("FinalData\\Articles\\", loader_cls=PyPDFLoader, show_progress=True, use_multithreading=True, glob='**/*.pdf')
pdf_docs = loader_pdf.load_and_split()



In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader_csv, loader_pdf])


In [ ]:
df_races

In [ ]:
df_drivers

In [ ]:
from langchain.chains.question_answering import load_qa_chain
prompt = "In the fantasy, which driver had the most DNFs?"
index.query(prompt)



In [3]:
from util import local_settings
#pip install --upgrade packaging

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)


from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
#agent = create_pandas_dataframe_agent(client, [df_drivers, df_constructors, df_races], verbose=True)


sk-LCAqduOz4rTSH8NhaGGhT3Bl


In [ ]:
agent.run("What was the average amount of points score by Verstappen in 2023? Give me the final result, and remember to ignore TotalPoints.")
